In [ ]:
pip install "unstructured[all-docs]" pillow pydantic lxml matplotlib

In [ ]:
!sudo apt-get update

In [ ]:
!sudo apt-get install poppler-utils

In [ ]:
!sudo apt-get install libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn

In [ ]:
!pip install unstructured-pytesseract
!pip install tesseract-ocr

In [ ]:
from unstructured.partition.pdf import partition_pdf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
raw_pdf_elements=partition_pdf(
    filename="file from your system",                   #mandatory,  upload your file
    strategy="hi_res",                                  #mandatory to use "hi_res" strategy
    extract_images_in_pdf=True,                         #mandatory to use set as True
    extract_image_block_types=["Image", "Table"],       #optional
    extract_image_block_to_payload=False,               #optional
    extract_image_block_output_dir="extracted_data"

)

In [ ]:
raw_pdf_elements

In [ ]:
from typing import Text
Header=[]
Footer=[]
Title=[]
NarrativeText=[]
Text=[]
ListItem=[]
for element in raw_pdf_elements:
  if "unstructured.documents.elements.Header" in str(type(element)):
    Header.append(str(element))
  elif "unstructured.documents.elements.Footer" in str(type(element)):
    Footer.append(str(element))
  elif "unstructured.documents.elements.Title" in str(type(element)):
    Title.append(str(element))
  elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
    NarrativeText.append(str(element))
  elif "unstructured.documents.elements.Text" in str(type(element)):
    Text.append(str(element))
  elif "unstructured.documents.elements.ListItem" in str(type(element)):
   ListItem.append(str(element))

In [ ]:
NarrativeText

In [ ]:
img=[]
for element in raw_pdf_elements:
  if "unstructured.documents.elements.Image" in str(type(element)):
    img.append(str(element))

In [ ]:
img

In [ ]:
img[0]

In [ ]:
img[6-8]

In [ ]:
raw_pdf_elements2=partition_pdf(
    filename="file.pdf",                                #mandatory,  upload your file
    strategy="hi_res",                                  #mandatory to use "hi_res" strategy
    extract_images_in_pdf=True,                         #mandatory to use set as True
    extract_image_block_types=["Image", "Table"],       #optional
    extract_image_block_to_payload=False,               #optional
    extract_image_block_output_dir="extracted_data"

)

In [ ]:
raw_pdf_elements2

In [ ]:
img=[]
for element in raw_pdf_elements2:
  if "unstructured.documents.elements.Image" in str(type(element)):
    img.append(str(element))

In [ ]:
img

In [ ]:
tab=[]
for element in raw_pdf_elements2:
  if "unstructured.documents.elements.Table" in str(type(element)):
    tab.append(str(element))

In [ ]:
tab

In [ ]:
tab[0]

In [ ]:
NarrativeText=[]
for element in raw_pdf_elements2:
  if "unstructured.documents.elements.NarrativeText" in str(type(element)):
    NarrativeText.append(str(element))

In [ ]:
NarrativeText

In [ ]:
ListItem=[]
for element in raw_pdf_elements2:
  if "unstructured.documents.elements.ListItem" in str(type(element)):
    ListItem.append(str(element))

In [ ]:
ListItem

In [ ]:
!pip install langchain_core

In [ ]:
!pip install google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 langchain-google-genai
!pip install langchain
!pip install chromadb
!pip install langchain_community


In [ ]:
len(tab)

In [ ]:
len(img)

In [ ]:
from langchain_core.output_parsers import StrOutputParser  # same as before
from langchain_core.prompts import ChatPromptTemplate      # same as before
from langchain_google_genai import ChatGoogleGenerativeAI  # Gemini instead of OpenAI


Table Summary

In [ ]:
#prompt
prompt_text="""You are an assistant tasked with summarizing tables for retrieval.\
These summaries will be embedded and used to retrieve the raw table elements.\
Give a concise summary of the table that is well optimized for retrieval.\ Table {element}"""

In [ ]:
prompt= ChatPromptTemplate.from_template(prompt_text)

In [ ]:
import os
from google.colab import userdata
# Get your Gemini API key securely
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
# Text summary chain with Gemini
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # or "gemini-1.5-pro" for better reasoning
    temperature=0,
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

In [ ]:
summarize_chain={"element":lambda x: x} | prompt | model | StrOutputParser()

In [ ]:
table_summaries=[]

In [ ]:
table_summaries=summarize_chain.batch(tab,{"max_concurrency": 5})

In [ ]:
table_summaries[0]

Text Summary

In [ ]:
#prompt
prompt_text="""You are an assistant tasked with summarizing text for retrieval.\
These summaries will be embedded and used to retrieve the raw text elements.\
Give a concise summary of the table or text that is well optimized for retrieval.text: {element}"""

In [ ]:
prompt= ChatPromptTemplate.from_template(prompt_text)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ✅ Use a valid model name:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

prompt = PromptTemplate(
    input_variables=["element"],
    template="{element}"
)

summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()


In [ ]:
import time
import random

def safe_batch_summarize(texts, chain, max_requests_per_minute=15, max_retries=3):
    results = []
    delay = 60.0 / max_requests_per_minute

    for i, t in enumerate(texts):
        for attempt in range(max_retries):
            try:
                result = chain.invoke({"element": t})
                results.append(result)
                break
            except Exception as e:
                print(f"[{i}] Attempt {attempt+1}/{max_retries} failed: {e}")
                if attempt < max_retries - 1:
                    wait = delay * (attempt + 1) + random.uniform(1, 3)
                    print(f"Retrying in {wait:.1f}s...")
                    time.sleep(wait)
                else:
                    results.append(f"[ERROR] Failed to summarize: {e}")
        time.sleep(delay + random.uniform(0, 1))

    return results


In [ ]:
# ✅ Use safe batching
text_summaries = safe_batch_summarize(Text, summarize_chain)
text_summaries


In [ ]:
text_summaries

In [ ]:
import base64
import os
from langchain_core.messages import HumanMessage

In [ ]:
def encode_image(image_path):
  """Getting the base64 string"""
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

def image_summarize(img_base64, prompt):
    """Summarize an image using Gemini"""
    chat = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",  # or "gemini-1.5-pro" for better quality
        max_output_tokens=1024
    )

    msg = chat.invoke(
        [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": f"data:image/jpeg;base64,{img_base64}"}
                ]
            }
        ]
    )

    return msg.content  # returns the text output


In [ ]:
import os
import base64

def encode_image(image_path):
    """Getting the base64 string from a file"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images.
    path: folder containing .jpg files OR a single image path OR a list of image paths
    """
    img_base64_list = []
    image_summaries = []

    prompt = """You are an assistant tasked with summarizing images for retrieval.
These summaries will be embedded and used to retrieve the raw image.
Give a concise summary of the image that is well optimized for retrieval."""

    # Determine what type of input path is
    if os.path.isdir(path):
        image_files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith(".jpg")]
    elif isinstance(path, list):
        image_files = path
    else:  # single image file
        image_files = [path]

    for img_path in image_files:
        base64_image = encode_image(img_path)
        img_base64_list.append(base64_image)
        image_summaries.append(image_summarize(base64_image, prompt))

    return img_base64_list, image_summaries


In [ ]:
#fpath="/content/extracted_data/"

In [ ]:
fpath = "/content/extracted_data/figure-11-13.jpg"  # single image file
img_base64_list, image_summaries = generate_img_summaries(fpath)

print("Summary:", image_summaries[0])


In [ ]:
image_summaries

In [ ]:
img_base64_list

In [ ]:
import uuid
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

def create_multi_vector_retriever(vectorstore, text_summaries, texts, table_summaries, tables, image_summaries, images):
    """
    Create retriever that indexes summaries, but returns raw images or texts
    """
    # Initialize the storage layer
    store = InMemoryStore()
    id_key = "doc_id"

    # Create the multi-vector retriever
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )

    # Helper function to add documents to the vectorstore and docstore
    def add_documents(retriever, doc_summaries, doc_contents):
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
        summary_docs = [
            Document(page_content=s, metadata={id_key: doc_ids[i]})
            for i, s in enumerate(doc_summaries)
        ]
        retriever.vectorstore.add_documents(summary_docs)
        retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

    # Add texts, tables, and images
    if text_summaries:
        add_documents(retriever, text_summaries, texts)
    if table_summaries:
        add_documents(retriever, table_summaries, tables)
    if image_summaries:
        add_documents(retriever, image_summaries, images)

    return retriever


# Use Gemini embeddings instead of OpenAI embeddings
vectorstore = Chroma(
    collection_name="mm_rag",
    embedding_function=GoogleGenerativeAIEmbeddings(model="models/embedding-001")
)

# Create retriever with correct variable names
retriever_multi_vector_img = create_multi_vector_retriever(
    vectorstore,
    text_summaries,
    Text,              # Text variable you used earlier
    table_summaries,
    tab,               # ✅ FIXED: use tab (your actual table data variable)
    image_summaries,
    img_base64_list
)


In [ ]:
retriever_multi_vector_img

In [ ]:
import io
import re
from Ipython.display import HTML, display
from PIL import Image

In [ ]:
def plt_img_base64(img_base64):
  """ Display base64 encoded string as image"""
  #create an html img tag with the base64 string as source
  image_html=f'<img src="data:image/jpeg;base64,{img_base64}"/>'
  #Display the image by rendering with HTML
  display(HTML(image_html))

In [ ]:
plt_img_base64(img_base64_list[1])

In [ ]:
image_summaries[1]

In [ ]:
def looks_like_base64(sb):
  """Check if the string looks like base64"""
  return re.match("^[A-Za-z0-9+/]+[=](0,2)$", sb) is not None

In [ ]:
def is_image_data(b64data):
"""
Check if the base64 data is an image by looking at the start of the data
"""
image_signatures = {

  b"\xFF\xDB\xFF": "jpg",
  b"\x89\x50\x4E\x47\x80\x84\x14\x0A": "png",
  b"\x47x49\x46\x38": "gif",
  b"\x52\x49\x46\x46": "webp",
}

try:
  header =base64.b64decode(b64data) [:8] #Decode and get the first & bytes
  for sig, format in image_signatures.items():
    if header.startswith(sig):
      return True
  return False
except Exception:
  return False

In [ ]:
def resize_base64_image(base64_string, size=(128, 128)):
"""
Resize an image encoded as a Base64 string
"""
#Decode the Base64 string

img_data =base64.b64decode(base64_string)
img = Image.open(io. BytesIO(img_data))

#Resize the image
resized_img =img.resize(size, Image. LANCZOS)

#Save the resized image to a bytes buffer
buffered =io.BytesIO()
resized_ing.save(buffered, format=img.format)

#Encode the resized image to Base64
return base64.b64encode(buffered.getvalue()).decode("utf-8")

In [ ]:
def split_image_text_types(docs):
  """
  Split base64-encoded images and texts
  """
  b64_images = []
  texts = []
  for doc in docs:
    #Check if the document is of type Document and extract page_content if so
    if isinstance(doc, Document):
      doc=doc.page_content
    if looks_like_base64(doc) and is_image_data(doc):
      doc=resize_base64_image(doc, size=(1300, 600))
      b64_images.append(doc)
    else:
      texts.append(doc)
  print(b64_images)
  print(texts)

  return {"images": b64_images, "texts": texts}

In [ ]:
def img_prompt_func(data_dict):
  """Join the context into a single string"""
  print(data_dict)
  formatted_texts = "\n".join(data_dict["context"] ["texts"])
  messages = []
  #Adding image(s) to the messages if present
  if data_dict["context"]["images"]:
    for image in data_dict["context"]["images"]:
      image_message = {
          "type": "image_url",
          "image_url": {"url": f"data:image/jpeg;base64, {image}"}
      }
      messages.append(image_message)
  #Adding the text for analysis
  text_message={
      "type": "text",
      "text": (
           "You are a helpful assistant.\n"
           "You will be given a mixed info(s) .\n"
           "Use this information to provide relevant information to the user question.\n"
           f"User-provided question: {data_dict['question']}\n\n"
           "Text and / or tables: \n"
           f"(formatted_texts)"
      ),

  }
messages.append(text_message)
return [HumanMessage(content=messages)]

In [ ]:
from langchain_core.runnables import RunnableLambda , RunnablePassthrough

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

def multi_modal_rag_chain(retriever):            #main method
    """Multi-modal RAG chain"""
    # Multi-modal LLM
    model = ChatGoogleGenerativeAI(temperature=0, model="gemini-1.5-flash", max_tokens=1024)

    # RAG pipeline
    chain = (
        {
            "context": retriever | RunnableLambda(split_image_text_types),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(img_prompt_func)
        | model
        | StrOutputParser()
    )
    return chain


In [ ]:
# create RAG chain
chain_multimodal_rag= multi_modal_rag_chain(retriever_multi_vector_img)

In [ ]:
chain_multidal_rag

Check

In [ ]:
#check retrieval
query ="Why We combine a pre-trained retriever (Query Encoder Document Index) with a pre-trained seq2seq model (Generator) and fine-tune"
docs= retriever_multi_vector_ing.invoke(query)

In [ ]:
docs

In [ ]:
query="Open-Domain QA Test Scores. Scores. For TQA,\
left column uses the standard test set for Open-\
Domain QA, right column uses the TQA-Wiki\
test set. See Appendix D for further details."

In [ ]:
 docs =retriever_multi_vector_img.invoke(query)

In [ ]:
docs

In [ ]:
query="Models are trained with either 5 or 10 retrieved latent\
documents, and we do not observe significant differences in performance between them."

In [ ]:
retriever_multi_vector_img.invoke(query)

In [ ]:
#We get back relevant images
plt_img_base64(docs)

Query

In [ ]:
query="can you explain me this Left: NQ performance as more documents are retrieved. Center: Retrieval recall performance\
In NQ. Right: MS-MARCO Bleu-1 and Rouge-L as more documents are retrieved."


In [ ]:
query1="Explain any images/figures in the paper with Left: NQ performance as more documents are retrieved. Center: Retrieval recall performance\
 in NQ. Right: MS-MARCO Bleu-1 and Rouge-L as more documents are retrieved."

In [ ]:
#Run RAG chain
chain_multimodal_rag.invoke(query)

In [ ]:
#Run RAG chain
chain_multimodal_rag.invoke(query1)